In [3]:
# =============================================================================
# Python Audio Property Analyzer
#
# This script takes an audio file as input and analyzes its properties,
# including:
# 1. Estimated Tempo (in Beats Per Minute)
# 2. Estimated Pitch (Fundamental Frequency)
# 3. Spectral Centroid (a measure of the "brightness" of a sound)
# 4. Zero-Crossing Rate (related to the noisiness of a signal)
# 5. A Spectrogram Visualization (showing all frequencies over time)
#
# Usage:
# 1. Save the script as a Python file (e.g., analyze_audio.py).
# 2. Make sure you have an audio file (e.g., my_song.wav) in the same directory.
# 3. Change the `AUDIO_FILE` variable below to match your file's name.
# 4. Run from the terminal: python analyze_audio.py
# =============================================================================

import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
import soundfile as sf

# --- CONFIGURATION ---
# IMPORTANT: Change this to the path of your audio file.
# You can use any common format like .wav, .mp3, .flac, etc.
AUDIO_FILE = 'my_song.wav' # <--- CHANGE THIS

In [ ]:
def analyze_audio_properties(file_path):
    """
    Loads an audio file and analyzes its key properties.
    """
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' was not found.")
        print("Please make sure the audio file is in the same directory as the script,")
        print("or provide the full path to the file.")
        
        # As an example, create a dummy audio file if one doesn't exist
        print("Creating a dummy audio file named 'my_song.wav' for demonstration.")
        sr = 22050  # sample rate
        duration = 5
        frequency = 440  # A4 note
        t = np.linspace(0., duration, int(sr * duration))
        amplitude = np.iinfo(np.int16).max * 0.5
        data = amplitude * np.sin(2. * np.pi * frequency * t)
        
        
        sf.write(file_path, data.astype(np.int16), sr)
        print("Dummy file 'my_song.wav' created. Please run the script again.")
        return

    try:
        print(f"Loading '{file_path}'...")
        y, sr = librosa.load(file_path)

        # 2. Analyze Properties
        # --- Tempo (BPM) ---
        onset_env = librosa.onset.onset_detect(y=y, sr=sr)
        tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]

        # --- Pitch (Fundamental Frequency) ---
        # We use the Yin algorithm for pitch tracking
        f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
        # Get the average pitch where the sound is considered "voiced"
        avg_pitch = np.mean(f0[voiced_flag]) if np.any(voiced_flag) else 0

        # --- Spectral Centroid ("Brightness") ---
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        avg_spectral_centroid = np.mean(spectral_centroid)
        
        # --- Zero-Crossing Rate ("Noisiness") ---
        zcr = librosa.feature.zero_crossing_rate(y)[0]
        avg_zcr = np.mean(zcr)

        # 3. Print the results
        print("\n--- Audio Properties ---")
        print(f"Estimated Tempo: {tempo:.2f} BPM")
        print(f"Average Pitch (Fundamental Frequency): {avg_pitch:.2f} Hz")
        print(f"Average Spectral Centroid (Brightness): {avg_spectral_centroid:.2f} Hz")
        print(f"Average Zero-Crossing Rate (Noisiness): {avg_zcr:.4f}")
        print("------------------------\n")
        
        # 4. Visualize the Spectrogram
        print("Displaying spectrogram... Close the plot window to exit.")
        D = librosa.stft(y)  # Short-time Fourier transform
        S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
        
        plt.figure(figsize=(12, 8))
        librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='log')
        plt.colorbar(format='%+2.0f dB')
        plt.title(f'Log-Frequency Power Spectrogram for {os.path.basename(file_path)}')
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"An error occurred during analysis: {e}")
        print("Please ensure you have ffmpeg installed if you are using MP3 files.")



In [ ]:

if __name__ == '__main__':
    pass
    #analyze_audio_properties(AUDIO_FILE)